# Step 1: Setup and Initialization

This cell imports the necessary libraries, sets up the project paths, and authenticates with Google Earth Engine (GEE). Run this cell first to prepare the environment.

import ee
import geemap
import os
import sys
from datetime import datetime, timedelta
from pathlib import Path

def initialize_gee():
    """Authenticates and initializes the Google Earth Engine library."""
    try:
        # A simple check to see if GEE is already initialized.
        ee.Image.constant(0).getInfo()
        print('✅ GEE is already authenticated and initialized.')
    except Exception:
        print('🔑 Authenticating and initializing GEE...')
        try:
            ee.Authenticate()
            ee.Initialize()
            print('✅ GEE authenticated and initialized successfully!')
        except Exception as e:
            print(f'❌ GEE initialization failed: {e}')
            # In a notebook, we might not want to exit, just raise the error
            raise e

# Run the initialization
initialize_gee()

# --- Configuration ---
# Define project paths. Assumes the notebook is in the 'notebooks' directory.
project_root = Path.cwd().parent
download_dir = project_root / 'data' / 'downloaded_tiles'
download_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📂 Files will be saved to: {download_dir}")

# Step 2: Define Download Parameters

This cell defines the geographical area of interest (Uzbekistan) and the time periods for which we want to download satellite imagery. You can customize the `date_ranges` dictionary to add or change the periods.

# Define the Area of Interest (AOI) for Uzbekistan
uzbekistan_bounds = ee.Geometry.Rectangle([55.9, 37.2, 73.2, 45.6])

# Define date ranges for the tiles you want to download
today = datetime.now()
date_ranges = {
    'recent_3_months': (
        (today - timedelta(days=90)).strftime('%Y-%m-%d'),
        today.strftime('%Y-%m-%d')
    ),
    'summer_2023': ('2023-06-01', '2023-08-31'),
    'winter_2023_2024': ('2023-12-01', '2024-02-29'),
}

print("🗓️  Date ranges for download:")
for period, (start, end) in date_ranges.items():
    print(f"   - {period}: {start} to {end}")

# Step 3: Process and Download Tiles

This is the main processing cell. It iterates through each defined period, finds the least cloudy Landsat 8 image, and downloads it as a GeoTIFF file to the `data/downloaded_tiles` directory.

def get_least_cloudy_image(collection, region):
    """
    Filters an ImageCollection to get the single least cloudy image.
    """
    return collection.sort('CLOUD_COVER').first().clip(region)

def download_gee_tile(image, region, filename, scale=30):
    """
    Downloads a GEE Image to a local GeoTIFF file.
    """
    print(f"🛰️  Starting download for: {os.path.basename(filename)}")
    try:
        geemap.ee_export_image(
            image,
            filename=str(filename), # geemap expects a string path
            scale=scale,
            region=region,
            file_per_band=False  # Export as a single multi-band GeoTIFF
        )
        print(f"✅ Successfully downloaded: {filename}")
    except Exception as e:
        print(f"❌ Error downloading {os.path.basename(filename)}: {e}")

# --- Processing ---
# Base Landsat 8 Collection 2, Surface Reflectance
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(uzbekistan_bounds)

for period, (start_date, end_date) in date_ranges.items():
    print(f"\n🔎 Processing period: {period} ({start_date} to {end_date})")
    
    # Filter the collection for the specific date range and low cloud cover
    period_collection = landsat_collection \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20))

    image_count = period_collection.size().getInfo()
    if image_count == 0:
        print(f"   - ⚠️ No images found for this period. Skipping.")
        continue
    
    print(f"   - ✅ Found {image_count} images for this period.")

    # Get the best (least cloudy) image from the filtered collection
    best_image = get_least_cloudy_image(period_collection, uzbekistan_bounds)
    
    # Select and scale the optical bands for analysis
    output_image = best_image.select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']) \
                             .multiply(0.0000275).add(-0.2)

    # Define the output filename and download the tile
    output_filename = download_dir / f'uzbekistan_tile_{period}.tif'
    download_gee_tile(output_image, uzbekistan_bounds, output_filename, scale=30)

print("\n🎉 All download tasks are complete.")